In [3]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [4]:
jack = Customer('Park', 100)
jack

Customer(name='Park', fidelity=100)

In [5]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [6]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [7]:
class Promotion(ABC):

    @abstractmethod
    def discount(self, order):
        pass

In [8]:
class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [10]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart=[LineItem('banana', 4, .5),
      LineItem('apple', 10, 1.5),
      LineItem('watermellon', 5, 5.0)]

In [11]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [13]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [14]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [15]:
Long_cart = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [16]:
Order(joe, Long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [17]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [18]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [22]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [23]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [24]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [25]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [27]:
Order(joe, Long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

In [28]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

In [32]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [33]:
Order(ann, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [34]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [35]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'ABC', '_i3', 'abstractmethod', 'namedtuple', 'Customer', '_i4', 'jack', '_4', '_i5', 'LineItem', '_i6', 'Order', '_i7', 'Promotion', '_i8', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i9', '_i10', 'joe', 'ann', 'cart', '_i11', '_11', '_i12', '_12', '_i13', 'banana_cart', '_i14', '_14', '_i15', 'Long_cart', '_i16', '_16', '_i17', '_17', '_i18', '_i19', '_i20', '_20', '_i21', '_21', '_i22', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', '_i27', '_27', '_i28', 'promos', 'best_promo', '_i29', '_29', '_i30', '_30', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35'])

In [36]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [37]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [38]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [39]:
@deco
def target():
  print('running target()')

target()

running inner()


In [40]:
target

<function __main__.deco.<locals>.inner()>

In [42]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7d819a770670>)
running register(<function f2 at 0x7d819a770700>)
running main()
registry -> [<function f1 at 0x7d819a770670>, <function f2 at 0x7d819a770700>]
running f1()
running f2()
running f3()


In [44]:
def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [45]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [46]:
b=6
f1(3)

3
6


In [47]:
b=6
def f2(a):
  print(a)
  print(b)
  b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [48]:
b=6
def f3(a):
  global b
  print(a)
  print(b)
  b=9

f3(3)

3
6


In [49]:
b

9

In [50]:
f3(3)

3
9


In [51]:
b=30
f3(3)

3
30


In [52]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [53]:
avg = Averager()
avg(10)

10.0

In [54]:
avg(11)

10.5

In [56]:
avg(12)

11.0

In [57]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

In [58]:
avg = make_averager()
avg(10)

10.0

In [59]:
avg(11)

10.5

In [60]:
avg(12)

11.0

In [61]:
avg.__code__.co_varnames

('new_value', 'total')

In [62]:
avg.__code__.co_freevars

('series',)

In [63]:
avg.__closure__

(<cell at 0x7d819ac44e50: list object at 0x7d819a7e2580>,)

In [64]:
avg.__closure__[0].cell_contents

[10, 11, 12]